In [1]:
##this version

import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
def conv_block(inputs, filters, kernel_size, strides):
    conv_1 = layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(inputs)
    bn_1 = layers.BatchNormalization()(conv_1)
    std_relu = layers.ReLU()(bn_1)  # Use standard ReLU activation
    return std_relu

In [3]:
def relu6(x):
    return tf.keras.backend.relu(x, max_value=6)

In [4]:
def depthwise_separable_conv_block(inputs, filters, kernel_size, strides):
    depth_conv = layers.DepthwiseConv2D(kernel_size=kernel_size, strides=strides, padding='same')(inputs)
    depth_conv_bn = layers.BatchNormalization()(depth_conv)
    depth_conv_relu = layers.ReLU()(depth_conv_bn)  # Use standard ReLU activation
    conv_1 = layers.Conv2D(filters, 1, strides=1, padding='same')(conv_1)
    conv_bn = layers.BatchNormalization()(conv_bn)
    conv_relu = relu6(conv_bn)  # Use ReLU6 activation function
    return conv_relu

In [5]:
def inverted_residual_block(inputs, expansion, filters, kernel_size, strides, use_residual=True):
    input_channels = inputs.shape[-1]

    # Expansion phase
    expand_conv = layers.Conv2D(expansion * input_channels, 1, padding='same')(inputs)
    expand_bn = layers.BatchNormalization()(expand_conv)
    expand_relu6 = relu6(expand_bn)  # Use ReLU6 activation function

    # Depthwise Convolution
    depth_conv_1 = layers.DepthwiseConv2D(kernel_size, strides=strides, padding='same')(expand_relu6)
    depth_bn = layers.BatchNormalization()(depth_conv_1)
    depth_relu6 = relu6(depth_bn)  # Use ReLU6 activation function

    # Projection phase
    project_conv = layers.Conv2D(filters, 1, padding='same')(depth_relu6)
    project_bn = layers.BatchNormalization()(project_conv)

    if use_residual and strides == 1 and input_channels == filters:
        res_add = layers.Add()([inputs, project_bn])
        return res_add

    return project_bn

In [6]:
def MobileNetV2(input_shape, number_of_class,activation):
    input_layer = tf.keras.Input(shape=input_shape)

    conv1 = conv_block(inputs=input_layer, filters=32,kernel_size= (3,3), strides=2)

    inv_res1 = inverted_residual_block(inputs=conv1, expansion=1, filters=16, kernel_size=(3,3), strides=1)

    inv_res2 = inverted_residual_block(inputs=inv_res1, expansion=6, filters=24, kernel_size=(3,3), strides=2)
    inv_res3 = inverted_residual_block(inputs=inv_res2, expansion=6, filters=24, kernel_size=(3,3), strides=1)

    inv_res4 = inverted_residual_block(inputs=inv_res3, expansion=6, filters=32, kernel_size=(3,3), strides=2)
    inv_res5 = inverted_residual_block(inputs=inv_res4, expansion=6, filters=32, kernel_size=(3,3), strides=1)
    inv_res6 = inverted_residual_block(inputs=inv_res5, expansion=6, filters=32, kernel_size=(3,3), strides=1)

    inv_res7 = inverted_residual_block(inputs=inv_res6, expansion=6, filters=64, kernel_size=(3,3), strides=2)
    inv_res8 = inverted_residual_block(inputs=inv_res7, expansion=6, filters=64, kernel_size=(3,3), strides=1)
    inv_res9 = inverted_residual_block(inputs=inv_res8, expansion=6, filters=64, kernel_size=(3,3), strides=1)
    inv_res10 = inverted_residual_block(inputs=inv_res9, expansion=6, filters=64, kernel_size=(3,3), strides=1)

    inv_res11 = inverted_residual_block(inputs=inv_res10, expansion=6, filters=96, kernel_size=(3,3), strides=1)
    inv_res12 = inverted_residual_block(inputs=inv_res11, expansion=6, filters=96, kernel_size=(3,3), strides=1)
    inv_res13 = inverted_residual_block(inputs=inv_res12, expansion=6, filters=96, kernel_size=(3,3), strides=1)

    inv_res14 = inverted_residual_block(inputs=inv_res13, expansion=6, filters=160, kernel_size=(3,3), strides=2)
    inv_res15 = inverted_residual_block(inputs=inv_res14, expansion=6, filters=160, kernel_size=(3,3), strides=1)
    inv_res16 = inverted_residual_block(inputs=inv_res15, expansion=6, filters=160, kernel_size=(3,3), strides=1)

    inv_res17 = inverted_residual_block(inputs=inv_res16, expansion=6, filters=320, kernel_size=(3,3), strides=1)

    conv2 = conv_block(inputs=inv_res17, filters=1280,kernel_size=(1,1) , strides=1)

    global_avg_pool = layers.GlobalAveragePooling2D()(conv2)

    output_layer = layers.Dense(units=number_of_class, activation=activation)(global_avg_pool)

    model = models.Model(inputs=input_layer, outputs=output_layer)

    return model

# Example usage with 3-channel RGB images and 1000 classes (Imagenet):
input_shape = (224, 224, 3)
number_of_class = 20
activation = 'softmax'

mobilenetv2_model = MobileNetV2(input_shape, number_of_class,activation)
mobilenetv2_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 112, 112, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             